In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import fft
from h5py import File

In [2]:
from KSVD import *
from ResultProcess import PSNR
from ImgProcess import *

In [3]:
filePath = '../Images/lena.tif'
size = 18
sparsity = 6
k = 3

In [ ]:
_imgClear = plt.imread('../Images/McM images/' + fileName + '.tif') / 255
_imgNoisy = File('../Images/McM images/' + fileName + '_noise.mat')
_imgNoisy = [_channel.T / 255 for _channel in _imgNoisy['u_n']]

In [ ]:
def Learning(fileName, size, sparsity, k):
    

    numDicts = int(np.ceil(size * k**0.5))
    len_dict = 2 + (size + 2) * numDicts
    img_dict = np.ones((len_dict, len_dict, 3))

    # learn the dictionary
    mD = fft.dct(np.eye(size**2 * k), norm='ortho')[: size**2, :]
    _, mD = KSVD(channel, size, sparsity, 100, mD=mD)

    # denoise the image using the learned dictionary
    # denoise the channel: use max_iter=1 only learning once for the sparse coding
    channel_denoised, _ = KSVD(channel_noisy, size, 1, eps=1e-7, mD=mD)
    # calculate the PSNR value
    psnr[color] = PSNR(channel, channel_denoised)

    # normalize the channel
    channel_denoised = (channel_denoised - channel_denoised.min()) / (channel_denoised.max() - channel_denoised.min())
    img_denoised.append(channel_denoised)

    # save the dictionary
    mDD = (mD - mD.min()) / (mD.max() - mD.min())
    for j in range(mDD.shape[1]):
        atom = mDD[:, j].reshape((size, size))
        idx = np.unravel_index(j, (numDicts, numDicts))
        x = idx[0] * (size + 2) + 2
        y = idx[1] * (size + 2) + 2
        img_dict[x:x + size, y:y + size, i] = atom